# Fit supernovae with SNooPy in batch

Python 3.7 Anaconda

NOTES:

- Probably I have a bug because when I specify to fit all the optical+nir
bands by listing all the bands explicitly, then with RAISIN-1 I couldn't
fit any supernova. However, when I don't specify expliclity all the bands
(then my script consider this as to fit all the bands), then I can
fit all the supernovae with no issue. Check this.

In [1]:
from snpy import *
from matplotlib import pyplot as plt
import numpy as np
import datetime # Get the current date and time
import os

#----------------------
%cd /Users/arturo/Dropbox/Research/Codes_Mains/Python/MyFunctions/github/MyPyFuns/SNeIa/
import mysnpyfunc # My snoopy functions

%cd /Users/arturo/Dropbox/Research/Articulos/18_RAISINs/Codes/github/RAISIN/

#----------------------
# import random # To compute k-corr uncertainties
# import json # To save the simulated mag and k-corr uncertainties.

# from scipy.integrate import quad as intquad # To integrate
# import matplotlib as plt

# import glob # To read the files in my directory

# %pylab qt 
# For CANOPY python: to show the plots in a separated Windows instead of inline.
# If used, then I don't have to put the instruction "plt.close()" at
# the end of the plot because it will show the figure and instantaneously 
# close the windows too.

#--------------------------------------------------------60
code_created_by = 'Arturo_Avelino'
# On date: '2019.06.17' (yyyy.mm.dd)
code_name = 'Fit_with_Snoopy_InBatch_MAIN.ipynb'
code_version = '0.1.4'
code_last_update = '2019.08.14'

/Users/arturo/Dropbox/Research/Codes_Mains/Python/MyFunctions/github/MyPyFuns/SNeIa
/Users/arturo/Dropbox/Research/Articulos/18_RAISINs/Codes/github/RAISIN


In [2]:
##############################################################################80

### Read the list of SNe to fit

Change to the directory where the LCs files are located:

In [3]:
dir_data = '/Users/arturo/Documents/Research/Workplace/raisin/raisin1/\
2019_08_05__v0_1_1/data_2019_02_28/'

os.chdir(dir_data)

In [ ]:
# cd /Users/arturo/Documents/Research/Workplace/raisin/raisin1/2019_08_05__v0_1_1/data  

In [4]:
ls

PScA470041_snoopy.dat  PScE510457_snoopy.dat  PScJ550202_snoopy.dat
PScA470110_snoopy.dat  PScF520062_snoopy.dat  PScJ560027_snoopy.dat
PScA470240_snoopy.dat  PScF520107_snoopy.dat  PScJ560054_snoopy.dat
PScB480464_snoopy.dat  PScF520188_snoopy.dat  PScK450082_snoopy.dat
PScB480794_snoopy.dat  PScG530251_snoopy.dat  PScK450339_snoopy.dat
PScC490037_snoopy.dat  PScH540087_snoopy.dat  about.txt
PScC490521_snoopy.dat  PScH540118_snoopy.dat  list_SNe.txt
PScD500100_snoopy.dat  PScJ440005_snoopy.dat
PScD500301_snoopy.dat  PScJ440236_snoopy.dat


In [5]:
pwd

'/Users/arturo/Documents/Research/Workplace/raisin/raisin1/2019_08_05__v0_1_1/data_2019_02_28'

In [6]:
list_sne = np.genfromtxt('list_SNe.txt', dtype=['S22', float])
len(list_sne)

23

In [ ]:
# list_sne[0][0]

In [ ]:
#-----------------------------------------------------------------------------80

Define the directory where I'll save the data:

In [7]:
dir_save_output = '/Users/arturo/Documents/Research/Workplace/raisin/raisin1/\
2019_08_05__v0_1_1/fits/current/'

if not os.path.exists(dir_save_output):
    os.makedirs(dir_save_output)

In [ ]:
#-----------------------------------------------------------------------------80

In [ ]:
# t_Bmax

In [14]:
str(list_sne[i1][0][2:-1])

"b'cK450339_snoopy.da'"

In [13]:
list_sne[i1]

(b'PScK450339_snoopy.dat', 56251.348)

In [16]:
list_sne[i1][0]

b'PScK450339_snoopy.dat'

In [17]:
type(list_sne[i1][0])

numpy.bytes_

In [18]:
np.str(list_sne[i1][0])

"b'PScK450339_snoopy.dat'"

In [19]:
list_sne[i1][0].decode('UTF-8')

'PScK450339_snoopy.dat'

In [20]:
type(list_sne[i1][0].decode('UTF-8'))

str

### Main loop

In [21]:
count_sn = 0
count_sn_errors = 0
fail_sne = []

textfile_1 = open(dir_save_output+'Settings_fits.txt', 'w')
textfile_2 = open(dir_save_output+'Failed_fits.log', 'w')

textfile_1.write('#       Settings of the fits\n')
textfile_1.write('# Directory where the data is located\n')
textfile_1.write('# %s\n'%dir_data)

textfile_2.write('# Failed supernovae to be fitted:\n')

#--------------------------------------------------------60

# Define the header text
now = datetime.datetime.now() # Read the time and date right now
text_timenow = now.strftime("%Y.%m.%d (yyyy.mm.dd); %H:%M hrs (ET).")
text_Date   = '# On date: %s\n'%text_timenow
text_Author = '# Data table created by: %s.\n'%code_created_by
text_script = '# Script used: %s (version %s | last update: %s)\n'%(
    code_name, code_version, code_last_update)
text_line = '#'+'-'*57 + '60\n'

# Write the header text
textfile_1.write(text_line);
textfile_1.write(text_Author); textfile_1.write(text_Date);
textfile_1.write(text_script);
textfile_1.write(text_line);

#--------------------------------------------------------60

for i1 in range(len(list_sne)):
    
    # snname = str(list_sne[i1][0])  # Python 2.7
    snname = list_sne[i1][0].decode('UTF-8')  # Python 3.7
    print('     ', snname, ": I'll try to fit it.")
    
    # When using a precomputed value for t_Bmax.
    t_Bmax = list_sne[i1][1]

    try:
        mysnpyfunc.snpyfit(snname, 
                         
           # Options: ('EBV_model', 'EBV_model2', 'max_model')
           model='EBV_model', 
                           
           mangled_kcorr=True,
                           
           # When using a precomputed value for t_Bmax, otherwise
           # comment the following line:
           Tmax = t_Bmax, 
                          
           #------------------------------
           # Note: If "bands_to_fit[]" is not specified, then all band in the
           # LC data file are fitted.
                           
           #      RAISIN-2
           # bands_to_fit=['g_des', 'r_des', 'i_des','z_des', 'f125w','f160w'],
           # bands_to_fit=['r_des', 'i_des', 'z_des', 'f125w', 'f160w'],
           # bands_to_fit=['g_des', 'r_des', 'i_des', 'z_des'],
           # bands_to_fit=['r_des', 'i_des', 'z_des'],
           # bands_to_fit=['f125w','f160w'],
           # bands_to_fit=['f160w'],
                      
           #      RAISIN-1
           bands_to_fit=['g_ps1', 'r_ps1', 'i_ps1', 'z_ps1', 'f125w', 'f160w'],
           # bands_to_fit=['r_ps1', 'i_ps1', 'z_ps1', 'f125w', 'f160w'],
           # bands_to_fit=['g_ps1', 'r_ps1', 'i_ps1', 'z_ps1'],
           # bands_to_fit=['r_ps1', 'i_ps1', 'z_ps1'],
              
           #------------------------------
                           
           num_char_trim=-11,
           debug = True,
           dir_save_output = dir_save_output
           )

    #     #-------------------------------------------
    #     # When using a precomputed value for t_Bmax.
    
    #     t_Bmax = list_sne[i1][1]
    #     mysnpyfunc.snpyfit(snname, Tmax = t_Bmax, 
    #        bands_to_fit=['z_des','f160w'],
    #        obs_rest_bands=[['f160w', 'J_K']],
    #        dir_save_output = dir_save_output)
    
        count_sn += 1
    
    except:
        print("\n# %s couldn't be fitted.\n"%snname)
        fail_sne += [snname]
        textfile_2.write('%s\n'%snname)
        count_sn_errors += 1
            
#--------------------------------------------------------60

text_100 = '\n--- %s SNe fitted | %s SNe not fitted ---\n'%(
    count_sn, count_sn_errors)
text_110 = '# Failed %s SNe:'%count_sn_errors

textfile_1.write(text_100); textfile_1.write(text_110)

print(text_100)
print(text_110)
print(fail_sne)

textfile_1.close()
textfile_2.close()

      PScA470041_snoopy.dat : I'll try to fit it.
#- Try to upload the file: PScA470041_snoopy.dat.
#- File uploaded sucessfully.
#- Try to specify the SNooPy model EBV_model.
#- SNooPy EBV_model model specified sucessfully.
#- PScA470041_snoopy.dat LC data: uploaded.

# PScA470041_snoopy.dat couldn't be fitted.

      PScA470110_snoopy.dat : I'll try to fit it.
#- Try to upload the file: PScA470110_snoopy.dat.
#- File uploaded sucessfully.
#- Try to specify the SNooPy model EBV_model.
#- SNooPy EBV_model model specified sucessfully.
#- PScA470110_snoopy.dat LC data: uploaded.
#- Fit: done.
#- 'snpy' file: saved.
#- Plot with no text: done and saved.
#- Plot with text: done and saved.
#- Plot overlay: done and saved.
#- Plot k-corrections: done and saved.
#- Plot filters: done and saved.
--------------------------------------------------------------------------------
SN  PScA470110
z = 0.3460          ra= 52.07060         dec=-28.62330 EBV_MW = 0.0083 +/- 0.0007 
Data in the following 

/Users/arturo/anaconda3/lib/python3.7/site-packages/snpy/model.py:547: RuntimeWarning: divide by zero encountered in power
  weights.append(sum(where(mask, power(err,-2), 0)))


#- File uploaded sucessfully.
#- Try to specify the SNooPy model EBV_model.
#- SNooPy EBV_model model specified sucessfully.
#- PScC490037_snoopy.dat LC data: uploaded.
#- Fit: done.
#- 'snpy' file: saved.
#- Plot with no text: done and saved.
#- Plot with text: done and saved.
#- Plot overlay: done and saved.
#- Plot k-corrections: done and saved.
#- Plot filters: done and saved.
--------------------------------------------------------------------------------
SN  PScC490037
z = 0.4219          ra=149.95720         dec=  0.81660 EBV_MW = 0.0190 +/- 0.0006 
Data in the following bands: f125w,  f160w,  g_ps1,  r_ps1,  i_ps1,  z_ps1,  
Fit results (if any):
   DM = 41.945  +/-  0.106  +/- 0.123 (sys)
   dm15 = 0.881  +/-  0.124  +/- 0.060 (sys)
   EBVhost = -0.093  +/-  0.060  +/- 0.060 (sys)
   Tmax = 56370.484  +/-  0.000  +/- 0.340 (sys)
#- Time, date, author, script: defined.
#- Summary text file: done and saved.
#- PScC490037 fitted with no issues.

      PScC490521_snoopy.dat : I'll

/Users/arturo/anaconda3/lib/python3.7/site-packages/snpy/model.py:547: RuntimeWarning: divide by zero encountered in power
  weights.append(sum(where(mask, power(err,-2), 0)))


#- File uploaded sucessfully.
#- Try to specify the SNooPy model EBV_model.
#- SNooPy EBV_model model specified sucessfully.
#- PScF520107_snoopy.dat LC data: uploaded.
#- Fit: done.
#- 'snpy' file: saved.
#- Plot with no text: done and saved.
#- Plot with text: done and saved.
#- Plot overlay: done and saved.
#- Plot k-corrections: done and saved.
#- Plot filters: done and saved.
--------------------------------------------------------------------------------
SN  PScF520107
z = 0.5190          ra=184.16100         dec= 47.81600 EBV_MW = 0.0156 +/- 0.0003 
Data in the following bands: f160w,  g_ps1,  r_ps1,  i_ps1,  z_ps1,  
Fit results (if any):
   DM = 41.948  +/-  0.108  +/- 0.456 (sys)
   dm15 = 0.816  +/-  0.112  +/- 0.060 (sys)
   EBVhost = 0.905  +/-  0.138  +/- 0.060 (sys)
   Tmax = 56456.012  +/-  0.000  +/- 0.340 (sys)
#- Time, date, author, script: defined.
#- Summary text file: done and saved.
#- PScF520107 fitted with no issues.

      PScF520188_snoopy.dat : I'll try to f

...

#- File uploaded sucessfully.
#- Try to specify the SNooPy model EBV_model.
#- SNooPy EBV_model model specified sucessfully.
#- PScG530251_snoopy.dat LC data: uploaded.
#- Fit: done.
#- 'snpy' file: saved.
#- Plot with no text: done and saved.
#- Plot with text: done and saved.
#- Plot overlay: done and saved.
#- Plot k-corrections: done and saved.
#- Plot filters: done and saved.
--------------------------------------------------------------------------------
SN  PScG530251
z = 0.4100          ra=334.63430         dec=  1.11490 EBV_MW = 0.0651 +/- 0.0027 
Data in the following bands: f125w,  f160w,  g_ps1,  r_ps1,  i_ps1,  z_ps1,  
Fit results (if any):
   DM = 42.196  +/-  0.067  +/- 0.099 (sys)
   dm15 = 0.886  +/-  0.080  +/- 0.060 (sys)
   EBVhost = 0.044  +/-  0.043  +/- 0.060 (sys)
   Tmax = 56492.703  +/-  0.000  +/- 0.340 (sys)
#- Time, date, author, script: defined.
#- Summary text file: done and saved.
#- PScG530251 fitted with no issues.

      PScH540087_snoopy.dat : I'll 

/Users/arturo/anaconda3/lib/python3.7/site-packages/snpy/model.py:547: RuntimeWarning: divide by zero encountered in power
  weights.append(sum(where(mask, power(err,-2), 0)))


#- File uploaded sucessfully.
#- Try to specify the SNooPy model EBV_model.
#- SNooPy EBV_model model specified sucessfully.
#- PScH540118_snoopy.dat LC data: uploaded.
#- Fit: done.
#- 'snpy' file: saved.
#- Plot with no text: done and saved.
#- Plot with text: done and saved.
#- Plot overlay: done and saved.
#- Plot k-corrections: done and saved.
#- Plot filters: done and saved.
--------------------------------------------------------------------------------
SN  PScH540118
z = 0.4765          ra=351.82260         dec= -0.04810 EBV_MW = 0.0297 +/- 0.0008 
Data in the following bands: f160w,  g_ps1,  r_ps1,  i_ps1,  z_ps1,  
Fit results (if any):
   DM = 42.212  +/-  0.093  +/- 0.160 (sys)
   dm15 = 0.900  +/-  0.084  +/- 0.060 (sys)
   EBVhost = 0.042  +/-  0.029  +/- 0.060 (sys)
   Tmax = 56519.262  +/-  0.000  +/- 0.340 (sys)
#- Time, date, author, script: defined.
#- Summary text file: done and saved.
#- PScH540118 fitted with no issues.

      PScJ440005_snoopy.dat : I'll try to f

/Users/arturo/anaconda3/lib/python3.7/site-packages/snpy/model.py:547: RuntimeWarning: divide by zero encountered in power
  weights.append(sum(where(mask, power(err,-2), 0)))


#- File uploaded sucessfully.
#- Try to specify the SNooPy model EBV_model.
#- SNooPy EBV_model model specified sucessfully.
#- PScJ440236_snoopy.dat LC data: uploaded.
#- Fit: done.
#- 'snpy' file: saved.
#- Plot with no text: done and saved.
#- Plot with text: done and saved.
#- Plot overlay: done and saved.
#- Plot k-corrections: done and saved.
#- Plot filters: done and saved.
--------------------------------------------------------------------------------
SN  PScJ440236
z = 0.4366          ra= 37.10190         dec= -4.42460 EBV_MW = 0.0227 +/- 0.0004 
Data in the following bands: f125w,  f160w,  g_ps1,  r_ps1,  i_ps1,  z_ps1,  
Fit results (if any):
   DM = 42.106  +/-  0.033  +/- 0.089 (sys)
   dm15 = 0.877  +/-  0.042  +/- 0.060 (sys)
   EBVhost = -0.083  +/-  0.038  +/- 0.060 (sys)
   Tmax = 56208.355  +/-  0.000  +/- 0.340 (sys)
#- Time, date, author, script: defined.
#- Summary text file: done and saved.
#- PScJ440236 fitted with no issues.

      PScJ550202_snoopy.dat : I'll

In [22]:
textfile_1.close(); textfile_1.close(); textfile_1.close();
textfile_2.close(); textfile_2.close(); textfile_2.close(); 

In [ ]:
##############################################################################80